In [1]:
import sys
from logHelper import logger

In [3]:
#Import the necessary modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

[2024-04-08 11:04:57,450: INFO: utils: NumExpr defaulting to 8 threads.]


In [4]:
#Import the necessary modules for NLP
! pip install nltk
import nltk

Unable to create process using 'C:\Users\Kartheek Kotha\.conda\envs\nlp\python.exe "C:\Users\Kartheek Kotha\.conda\envs\nlp\Scripts\pip-script.py" install nltk'


## Data Reading

During this exam, each test taker sees a Standardized Patient, a person trained to portray a clinical case. After interacting with the patient, the test taker documents the relevant facts of the encounter in a patient note. Each patient note is scored by a trained physician who looks for the presence of certain key concepts or features relevant to the case as described in a rubric. The goal of this competition is to develop an automated way of identifying the relevant features within each patient note, with a special focus on the patient history portions of the notes where the information from the interview with the standardized patient is documented.


### FEATURES
--------
143 rows<br>
3 columns:  
<ol>
 <li>feature_num : unique ID no. of each complaint/ observation, in increasing order, not continuous<br>143 unique vals</li>  
 <li>case_num : number of patients with that particular complaint/observation<br>10 unique vals</li>  
 <li>feature_text : complaint/observation - distinct<br>131 unique vals</li>
 </ol>

In [ ]:
#extact data from the features.csv file into a pandas dataframe
features_df = pd.read_csv('../data/nbme-score-clinical-patient-notes/features.csv')
print(features_df.head(5))
print(features_df.columns)
print(len(features_df))

   feature_num  case_num                                       feature_text
0            0         0  Family-history-of-MI-OR-Family-history-of-myoc...
1            1         0                 Family-history-of-thyroid-disorder
2            2         0                                     Chest-pressure
3            3         0                              Intermittent-symptoms
4            4         0                                        Lightheaded
Index(['feature_num', 'case_num', 'feature_text'], dtype='object')
143


In [ ]:
#unique values in each col of features.csv
for col in features_df.columns:
    print(col, features_df[col].nunique())

feature_num 143
case_num 10
feature_text 131



### PATIENT NOTES
--------
42146 rows<br>
3 columns:  
<ol>
 <li>pn_num : unique ID no. of each patient, increasing order<br>42146 unique vals
 </li>  
 <li>case_num : **idk what this represents**<br>10 unique vals</li>  
 <li>pn_history : detailed description of complaints and observations. Most patients also have details of PMH, PSHH, FH, Social History, Medications, Sexual history, POS, Family history, Allergies. A patient has either all, or some of these details filled, under either acronyms or varying subheadings.<br>42146 unique vals</li>
 </ol>

In [ ]:
patientnotes_df = pd.read_csv('../data/nbme-score-clinical-patient-notes/patient_notes.csv')
print(patientnotes_df.head(5))
print(patientnotes_df.columns)
print(len(patientnotes_df))


   pn_num  case_num                                         pn_history
0       0         0  17-year-old male, has come to the student heal...
1       1         0  17 yo male with recurrent palpitations for the...
2       2         0  Dillon Cleveland is a 17 y.o. male patient wit...
3       3         0  a 17 yo m c/o palpitation started 3 mos ago; \...
4       4         0  17yo male with no pmh here for evaluation of p...
Index(['pn_num', 'case_num', 'pn_history'], dtype='object')
42146


In [ ]:
#unique values in each col of patient_notes.csv
for col in patientnotes_df.columns:
    print(col, patientnotes_df[col].nunique())

pn_num 42146
case_num 10
pn_history 42146


### SAMPLE SUBMISSION
--------
5 rows<br>
2 columns:  
<ol>
 <li>id : submission number<br>5 unique vals
 </li>  
 <li>location : **idk what this represents**<br>3 unique vals, some NaN(blank) values</li>  

 </ol>

In [ ]:
samplesub_df = pd.read_csv('../data/nbme-score-clinical-patient-notes/sample_submission.csv')
print(samplesub_df.head(5))
print(samplesub_df.columns)
print(len(samplesub_df))
for col in samplesub_df.columns:
    print(col, samplesub_df[col].nunique())

          id         location
0  00016_000            0 100
1  00016_001              NaN
2  00016_002  200 250;300 400
3  00016_003              NaN
4  00016_004           75 110
Index(['id', 'location'], dtype='object')
5
id 5
location 3


### Train and Test

In [ ]:
train_df = pd.read_csv('../data/nbme-score-clinical-patient-notes/train.csv')
print(train_df.head)

<bound method NDFrame.head of               id  case_num  pn_num  feature_num  \
0      00016_000         0      16            0   
1      00016_001         0      16            1   
2      00016_002         0      16            2   
3      00016_003         0      16            3   
4      00016_004         0      16            4   
...          ...       ...     ...          ...   
14295  95333_912         9   95333          912   
14296  95333_913         9   95333          913   
14297  95333_914         9   95333          914   
14298  95333_915         9   95333          915   
14299  95333_916         9   95333          916   

                                     annotation              location  
0              ['dad with recent heart attcak']           ['696 724']  
1                 ['mom with "thyroid disease']           ['668 693']  
2                            ['chest pressure']           ['203 217']  
3          ['intermittent episodes', 'episode']  ['70 91', '176 183']

In [ ]:
print(train_df.columns)

Index(['id', 'case_num', 'pn_num', 'feature_num', 'annotation', 'location'], dtype='object')


In [ ]:
print(len(train_df))
print(train_df.shape)

14300
(14300, 6)


In [ ]:
train_df.head(1)

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']


In [ ]:
test_df = pd.read_csv('../data/nbme-score-clinical-patient-notes/test.csv')
test_df.head(5)

,id,case_num,pn_num,feature_num
0,00016_000,0,16,0
1,00016_001,0,16,1
2,00016_002,0,16,2
3,00016_003,0,16,3
4,00016_004,0,16,4


In [ ]:
test_df.columns

Index(['id', 'case_num', 'pn_num', 'feature_num'], dtype='object')

In [ ]:
#get patient notes for a particular patient with pn_num = 16 m case_num = 1
print(patientnotes_df[(patientnotes_df['pn_num'] == 16) & (patientnotes_df['case_num'] == 0)]['pn_history'].values[0])

HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. 
PMHx: none
Rx: uses friends adderrall
FHx: mom with "thyroid disease," dad with recent heart attcak
All: none
Immunizations: up to date
SHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms


In [ ]:
print(features_df[(features_df['feature_num'] == 0) & (features_df['case_num'] == 0)]['feature_text'].values[0])

Family-history-of-MI-OR-Family-history-of-myocardial-infarction


## Data Analysis